In [1]:
import numpy as np
import pandas as pd

In [2]:
# Read the CSV files
df1 = pd.read_csv('data_ComputersElectronicProducts.csv', skiprows=6, delimiter='\t')
df2 = pd.read_csv('data_DefenseCapitalGoods.csv', skiprows=6, delimiter='\t')
df3 = pd.read_csv('data_MotorVehiclesParts.csv', skiprows=6, delimiter='\t')
df4 = pd.read_csv('data_PrimaryMetals.csv', skiprows=6, delimiter='\t')

In [3]:
# Διαχωρίζουμε τη στήλη "Period,Value" σε δύο στήλες "Period" και "Value"
df1[['Period', 'Value']] = df1['Period,Value'].str.split(',', expand=True)
df2[['Period', 'Value']] = df2['Period,Value'].str.split(',', expand=True)
df3[['Period', 'Value']] = df3['Period,Value'].str.split(',', expand=True)
df4[['Period', 'Value']] = df4['Period,Value'].str.split(',', expand=True)

In [4]:
# Μετατροπή της στήλης "Value" σε αριθμητικές τιμές
df1['Value'] = pd.to_numeric(df1['Value'], errors='coerce')
df2['Value'] = pd.to_numeric(df2['Value'], errors='coerce')
df3['Value'] = pd.to_numeric(df3['Value'], errors='coerce')
df4['Value'] = pd.to_numeric(df4['Value'], errors='coerce')

In [5]:
# Αφαίρεση των γραμμών με απουσιάζουσες τιμές
df1 = df1.dropna()
df2 = df2.dropna()
df3 = df3.dropna()
df4 = df4.dropna()

In [6]:
R1 = (df1['Value'].values / df1['Value'].values[0]) - 1
R2 = (df2['Value'].values / df2['Value'].values[0]) - 1
R3 = (df3['Value'].values / df3['Value'].values[0]) - 1
R4 = (df4['Value'].values / df4['Value'].values[0]) - 1

In [7]:
R = [R1, R2, R3, R4]

In [8]:
def calculate_mean_relative_return(R):
    n = len(R)
    mean_R = []

    for i in range(n):
        mean_R_i = np.mean(R[i])
        mean_R.append(mean_R_i)

    return mean_R

mean_R = np.array(calculate_mean_relative_return(R))
mean_R

array([0.29283429, 0.38721167, 0.76660544, 0.62611171])

In [9]:
def calculate_covariance_matrix(R):
    num_variables = len(R)  # Number of variables
    M = np.zeros((num_variables, num_variables))

    for i in range(num_variables):
        for j in range(num_variables):
            cov_ij = np.cov(R[i], R[j])[0, 1]
            M[i, j] = cov_ij

    return M
# Call the function with your R values
M = calculate_covariance_matrix(R)
M

array([[ 0.07493161,  0.01645338, -0.00706299, -0.0259072 ],
       [ 0.01645338,  0.38679314,  0.11740771,  0.1254754 ],
       [-0.00706299,  0.11740771,  0.21395927,  0.13668301],
       [-0.0259072 ,  0.1254754 ,  0.13668301,  0.2187645 ]])

In [10]:
#Calculate objective function
def objective_function(x, mean_R, M, lambd):
    total_sum = np.sum(x)
    
    w= x / total_sum if total_sum != 0 else x
    
    sum1 = -np.dot(w, mean_R) 
    
    sum2 = np.sum(np.outer(w,w) * M)
    
    return sum1 + lambd*sum2

In [11]:
#Calculate gradient
def gradient(x, mean_R, M, lambd):
    def partial_derivative_x1(x, mean_R, M, lambd):
        total_sum = np.sum(x)
        part1 = - ( - ((mean_R[3] - mean_R[0]) * x[3] + (mean_R[2] - mean_R[0]) * x[2] + (mean_R[1] - mean_R[0]) * x[1])/total_sum**2)
        part2 = 2 * (M[0,0] * x[0] * (x[1]+x[2]+x[3])/(total_sum)**3 - M[0,1] * x[1]*(x[0]-x[1]-x[2]-x[3])/(total_sum)**3 - M[0,2] * x[2]*(x[0]-x[1]-x[2]-x[3])/(total_sum)**3 - M[0,3] * x[3]*(x[0]-x[1]-x[2]-x[3])/(total_sum)**3)
        part3 = 2 * (- M[3,3] * x[3]**2/(total_sum)**3 - M[2,2] * x[2]**2/(total_sum)**3 - M[1,1] * x[1]**2/(total_sum)**3)
        part4 = 4 * (- M[1,2] * x[1]*x[2]/(total_sum)**3 - M[1,3] * x[1]*x[3]/(total_sum)**3 - M[2,3] * x[2]*x[3]/(total_sum)**3)       
   
        return (part1 + lambd * (part2 + part3 + part4))

    def partial_derivative_x2(x, mean_R, M, lambd):
        total_sum = np.sum(x)
        part1 = - ( - ((mean_R[3] - mean_R[1]) * x[3] + (mean_R[2] - mean_R[1]) * x[2] + (mean_R[0] - mean_R[1]) * x[0])/total_sum**2)
        part2 = 2 * (M[1,1] * x[1] * (x[0]+x[2]+x[3])/(total_sum)**3 - M[1,0] * x[0]*(x[1]-x[0]-x[2]-x[3])/(total_sum)**3 - M[1,2] * x[2]*(x[0]-x[1]-x[2]-x[3])/(total_sum)**3 - M[1,3] * x[3]*(x[1]-x[0]-x[2]-x[3])/(total_sum)**3)
        part3 = 2 * (- M[3,3] * x[3]**2/(total_sum)**3 - M[2,2] * x[2]**2/(total_sum)**3 - M[0,0] * x[0]**2/(total_sum)**3)
        part4 = 4 * (- M[0,2] * x[0]*x[2]/(total_sum)**3 - M[0,3] * x[0]*x[3]/(total_sum)**3 - M[2,3] * x[2]*x[3]/(total_sum)**3)       

        return (part1 + lambd * (part2 + part3 + part4))

    def partial_derivative_x3(x, mean_R, M, lambd):
        total_sum = np.sum(x)
        part1 = - ( - ((mean_R[3] - mean_R[2]) * x[3] + (mean_R[1] - mean_R[2]) * x[1] + (mean_R[0] - mean_R[2]) * x[0])/total_sum**2)
        part2 = 2 * (M[2,2] * x[2] * (x[0]+x[1]+x[3])/(total_sum)**3 - M[2,0] * x[0]*(x[2]-x[0]-x[1]-x[3])/(total_sum)**3 - M[2,1] * x[1]*(x[2]-x[0]-x[1]-x[3])/(total_sum)**3 - M[2,3] * x[3]*(x[2]-x[0]-x[1]-x[3])/(total_sum)**3)
        part3 = 2 * (- M[3,3] * x[3]**2/(total_sum)**3 - M[1,1] * x[1]**2/(total_sum)**3 - M[0,0] * x[0]**2/(total_sum)**3)
        part4 = 4 * (- M[0,1] * x[0]*x[1]/(total_sum)**3 - M[0,3] * x[0]*x[3]/(total_sum)**3 - M[1,3] * x[1]*x[3]/(total_sum)**3)       
   
        return (part1 + lambd * (part2 + part3 + part4))

    def partial_derivative_x4(x, mean_R, M, lambd):
        total_sum = np.sum(x)
        part1 = - ((mean_R[3] - mean_R[2]) * x[2] + (mean_R[3] - mean_R[1]) * x[1] + (mean_R[3] - mean_R[0]) * x[0])/total_sum**2
        part2 = 2 * (M[3,3] * x[3] * (x[0]+x[1]+x[2])/(total_sum)**3 - M[3,0] * x[0]*(x[3]-x[0]-x[1]-x[2])/(total_sum)**3 - M[3,1] * x[1]*(x[3]-x[0]-x[1]-x[2])/(total_sum)**3 - M[3,2] * x[2]*(x[3]-x[0]-x[1]-x[2])/(total_sum)**3)
        part3 = 2 * (- M[2,2] * x[2]**2/(total_sum)**3 - M[1,1] * x[1]**2/(total_sum)**3 - M[0,0] * x[0]**2/(total_sum)**3)
        part4 = 4 * (- M[0,1] * x[0]*x[1]/(total_sum)**3 - M[0,2] * x[0]*x[2]/(total_sum)**3 - M[1,2] * x[1]*x[2]/(total_sum)**3)       

        return (part1 + lambd * (part2 + part3 + part4))

    # Επιστροφή ενός πίνακα NumPy αντί για ένα tuple
    return np.array([partial_derivative_x1(x, mean_R, M, lambd),
                     partial_derivative_x2(x, mean_R, M, lambd),
                     partial_derivative_x3(x, mean_R, M, lambd),
                     partial_derivative_x4(x, mean_R, M, lambd)])

In [12]:
#line search
#zoom
#bisection
#BFGS

In [13]:
def line_search_wolfe(f, g, x, d, c1=1e-5, c2=0.8, max_iter=100):
    alpha = 1  # Αρχικό μήκος βήματος
    phi0 = f(x)  # Αρχική τιμή της συνάρτησης
    alpha_max = 2  # Μέγιστο μήκος βήματος
    phi_prime0 = np.dot(g(x), d)  # Αρχική παράγωγος της συνάρτησης στην κατεύθυνση d
    alpha_prev = 0.0  # Προηγούμενο μήκος βήματος
    phi_prev = phi0  # Προηγούμενη τιμή της συνάρτησης
    
    for i in range(max_iter):
        x_new = x + alpha * d  # Υπολογισμός νέας θέσης
        phi = f(x_new)  # Υπολογισμός τιμής συνάρτησης στο νέο σημείο
        phi_prime = np.dot(g(x_new), d)  # Υπολογισμός παραγώγου συνάρτησης στο νέο σημείο

        # Συνθήκη Armijo
        if phi > phi0 + c1 * alpha * phi_prime0 or (i > 0 and phi >= phi_prev):
            return zoom(f, g, x, d, alpha_prev, alpha)

        # Συνθήκη Curvature
        if abs(phi_prime) <= -c2 * phi_prime0:
            return alpha

        # Συνθήκη για αρνητική παράγωγο
        if phi_prime >= 0:
            return zoom(f, g, x, d, alpha, alpha_prev)

        alpha_prev = alpha
        phi_prev = phi
        alpha *= 2.0  # Διπλασιασμός μήκους βήματος

        # Εφαρμογή ορίου για το μέγιστο μήκος βήματος
        if alpha > alpha_max:
            return alpha_max

    return alpha

In [12]:
def zoom(f, g, x, d, alpha_lo, alpha_hi, c1=1e-4, c2=0.9, max_iter=100):
    phi0 = f(x)  # Αρχική τιμή της συνάρτησης
    phi_prime0 = np.dot(g(x), d)  # Αρχική παράγωγος της συνάρτησης στην κατεύθυνση d

    for i in range(max_iter):
        alpha = 0.5 * (alpha_lo + alpha_hi)  # Υπολογισμός νέου μήκους βήματος
        x_new = x + alpha * d  # Υπολογισμός νέας θέσης
        phi = f(x_new)  # Υπολογισμός τιμής συνάρτησης στο νέο σημείο
        phi_prime = np.dot(g(x_new), d)  # Υπολογισμός παραγώγου συνάρτησης στο νέο σημείο

        # Συνθήκες Armijo και Curvature
        if phi > phi0 + c1 * alpha * phi_prime0 or phi >= f(x + alpha_lo * d):
            alpha_hi = alpha
        else:
            if abs(phi_prime) <= -c2 * phi_prime0:
                return alpha
            if phi_prime * (alpha_hi - alpha_lo) >= 0:
                alpha_hi = alpha_lo
            alpha_lo = alpha

    return alpha

In [12]:
def bisection(f, a, b, epsilon):
    k = 0
    stop = 0
    
    while stop == 0:
        x = 0.5 * (b + a)  # Υπολογισμός του μέσου του διαστήματος [a, b]
        fx = f(x)  # Υπολογισμός της τιμής της συνάρτησης στο x

        # Έλεγχος αν κάποιο στοιχείο του fx είναι μηδέν
        if np.any(fx == 0):
            stop = 1
        else:
            # Έλεγχος αν κάποιο στοιχείο του f(a) * fx είναι αρνητικό
            if np.any(f(a) * fx < 0):
                a_new = a
                b_new = x
            else:
                a_new = x
                b_new = b
                
            k += 1
            
            # Έλεγχος αν το νέο διάστημα είναι εντός του επιθυμητού tolerance (επιτυγχάνεται η σύγκλιση)
            if b_new - a_new < epsilon:
                stop = 1
        
        a = a_new
        b = b_new
    
    return 0.5 * (b + a)  # Επιστροφή του μέσου του τελικού διαστήματος ως εκτιμώμενη ρίζα

In [16]:
def bfgs_algorithm(x0, mean_R, M, lambd, max_iter=100, tol=1e-6):
    n = len(x0)
    x = x0.copy()
    B = np.eye(n)  # Initial approximation of the Hessian matrix
    g = gradient(x, mean_R, M, lambd)  # Calculate the gradient
    k = 0
    
    for _ in range(max_iter):
        d = -np.dot(np.linalg.pinv(B), g)  # Calculate the search direction
        
        alpha = line_search_wolfe(lambda x: objective_function(x, mean_R, M, lambd),
                                  lambda x: gradient(x, mean_R, M, lambd),
                                  x, d)  # Calculate the step size
        
        x_new = x + alpha * d  # Update the solution
        #Call Bisection
        x_new = np.array([bisection(lambda x: objective_function(x, mean_R, M, lambd), 0, 1, 1e-8) if xi < 0 or xi > 1 else xi for xi in x_new])
        # Υπολογισμός του νέου gradient
        g_new = gradient(x_new, mean_R, M, lambd)  # Calculate the new gradient
        # Υπολογισμός των διανυσμάτων s και y
        s = x_new - x
        y = g_new - g
        #Update BFGS
        denominator = np.dot(y, s)
        if denominator != 0.0:
            B = B - np.outer(np.dot(B, s), np.dot(s, B)) / denominator + np.outer(y, y) / np.dot(y, s)
        
        x = x_new
        
        k += 1
       
    return x

In [17]:
#Αρχικοποίηση
bounds = [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]
R = [R1, R2, R3, R4]
lambd = 1.5
num_initial_points = 10
results = []
used_initial_points = set()  # To keep track of used initial points

In [18]:
best_solution = None
best_objective_value = float('inf')
best_x0 = None

In [19]:
# Execute the algorithm from 10 independently and randomly selected initial points
for i in range(num_initial_points):
    while True:
        x0 = np.random.uniform(bounds[0][0], bounds[0][1], size=4)
        # Convert the array to a tuple for set comparison
        x0_tuple = tuple(x0)
        if x0_tuple not in used_initial_points:
            used_initial_points.add(x0_tuple)
            break
    
    result = bfgs_algorithm(x0, mean_R, M, lambd)
    results.append(result)
    
    # Calculate the total sum and weight vector
    total_sum = np.sum(result)
    w = result / total_sum 

    # Print information for each run
    print(f"Run {i + 1}:")
    print("Initial Point (x0):", x0)
    print("Solution:", result)
    print ("w", w)
    print("Expected Return:", np.dot(w, mean_R))
    print("Risk:", np.sum(np.outer(w, w) * M))
    objective_value = -np.dot(w, mean_R) + lambd * np.sum(np.outer(w, w) * M)
    print("Objective Function Value:", objective_value)
    
    print("\n")

    # Check if the current solution is better than the previous best
    if objective_value < best_objective_value:
        best_solution = result
        best_objective_value = objective_value
        best_x0 = x0

# Print information about the best solution
print("Best Solution:")
print("Initial Point (x0):", best_x0)
print("Solution:", best_solution)
best_w = best_solution / np.sum(best_solution) if np.sum(best_solution) != 0 else best_solution
print("Best w:", best_w)
print("Expected Return:", np.dot(best_w, mean_R))
print("Risk:", np.sum(np.outer(best_w, best_w) * M))
print("Objective Function Value:", best_objective_value)


Run 1:
Initial Point (x0): [0.40644959 0.8206277  0.93041458 0.89789864]
Solution: [0.47754794 0.59262359 1.         0.92883799]
w [0.15923522 0.19760644 0.33344342 0.30971492]
Expected Return: 0.5726807299998574
Risk: 0.11856909407492536
Objective Function Value: -0.39482708888746937


Run 2:
Initial Point (x0): [0.64492451 0.62667696 0.59968307 0.09749737]
Solution: [0.65967137 0.47073913 0.73217954 0.1937992 ]
w [0.3207911  0.22891538 0.35605105 0.09424247]
Expected Return: 0.5145343221413725
Risk: 0.0900089915551312
Objective Function Value: -0.3795208348086757


Run 3:
Initial Point (x0): [0.07131722 0.66559967 0.50325192 0.68885868]
Solution: [0.19053419 0.47445364 0.6386414  0.73393907]
w [0.09351058 0.23285288 0.31343313 0.36020342]
Expected Return: 0.5833535724766601
Risk: 0.138636928470019
Objective Function Value: -0.3753981797716316


Run 4:
Initial Point (x0): [0.8279663  0.55970801 0.50057618 0.23135807]
Solution: [0.82074971 0.43587832 0.6254829  0.31053459]
w [0.3743193

In [23]:
Bestgradient=gradient([0.20297396, 0.02024999, 0.78894057, 0.38454184], mean_R, M, lambd)
print("Best gradient:", Bestgradient )

Best gradient: [-0.01833365  0.1106412  -0.0156814   0.03406059]


In [21]:
#DOGLEG

In [13]:
def dogleg_bfgs(x0, mean_R, M, lambd, max_iter=100, tol=1e-6, delta=0.1):
    n = len(x0)
    x = x0.copy()
    B = np.eye(n)  # Initial approximation of the Hessian matrix
    g = gradient(x, mean_R, M, lambd)  # Calculate the gradient
    k = 0
    
    while np.linalg.norm(g) > tol and k < max_iter:
        d_B = -np.dot(np.linalg.pinv(B), g)
        d_U = -g * np.dot(g, g) / np.dot(g, np.dot(B, g))#Cauchy step
        
        if np.linalg.norm(d_B) <= delta:
            d = d_B  # Take the Newton direction
        elif np.linalg.norm(d_U) >= delta:
            d = (delta / np.linalg.norm(d_U)) * d_U  # Take the steepest descent direction
        else:
            a = np.linalg.norm(d_U) ** 2
            b = 2 * np.dot(d_U, d_B - d_U)
            c = np.linalg.norm(d_B - d_U) ** 2 - delta ** 2
            
            # Check if the value inside the square root is non-negative
            if b ** 2 - 4 * a * c < 0:
                t = 0  # Set t to 0 if the value is negative
            else:
                t = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)  # Compute the intersection point
            
            d = d_U + t * (d_B - d_U)  # Take the dogleg direction
        
            
        x_new = x + d  # Update the solution
        #Call Bisection
        x_new = np.array([bisection(lambda x: objective_function(x, mean_R, M, lambd), 0, 1, 1e-8) if xi < 0 or xi > 1 else xi for xi in x_new])
        g_new = gradient(x_new, mean_R, M, lambd)  # Calculate the new gradient
        

        #Calculate ρ 
        denominator_rho = -np.dot(g, d) - 0.5 * np.dot(d, np.dot(B, d))
        if denominator_rho != 0.0:
            rho = (objective_function(x, mean_R, M, lambd) - objective_function(x_new, mean_R, M, lambd)) / denominator_rho
        else:
            rho = 0.0
            
        if rho < 0.25:
            delta *= 0.25
        elif rho > 0.75 and np.linalg.norm(d) == delta:
            delta = min(2 * delta, 1.0)
        else:
            delta = delta
        
        if rho > 0.1:
            # BFGS update
            y = g_new - g
            s = x_new - x
            denominator = np.dot(y, s)
            if denominator != 0.0:
                B = B - np.outer(np.dot(B, s), np.dot(s, B)) / denominator + np.outer(y, y) / np.dot(y, s)
            x = x_new
      
        g = g_new
        k += 1
    
    return x

In [14]:
bounds = [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]
R = [R1, R2, R3, R4]
lambd = 1.5
num_initial_points = 10
results = []
used_initial_points = set()  # To keep track of used initial points

In [15]:
best_solution = None
best_objective_value = float('inf')
best_x0 = None

In [38]:
# Execute the algorithm from 10 independently and randomly selected initial points
for i in range(num_initial_points):
    while True:
        x0 = np.random.uniform(bounds[0][0], bounds[0][1], size=4)
        # Convert the array to a tuple for set comparison
        x0_tuple = tuple(x0)
        if x0_tuple not in used_initial_points:
            used_initial_points.add(x0_tuple)
            break
    
    result = dogleg_bfgs(x0, mean_R, M, lambd)
    results.append(result)
    # Calculate the total sum and weight vector
    total_sum = np.sum(result)
    w = result / total_sum 

    # Print information for each run
    print(f"Run {i + 1}:")
    print("Initial Point (x0):", x0)
    print("Solution:", result)
    print ("w", w)
    print("Expected Return:", np.dot(w, mean_R))
    print("Risk:", np.sum(np.outer(w, w) * M))
    objective_value = -np.dot(w, mean_R) + lambd * np.sum(np.outer(w, w) * M)
    print("Objective Function Value:", objective_value)
    print("\n")

    # Check if the current solution is better than the previous best
    if objective_value < best_objective_value:
        best_solution = result
        best_objective_value = objective_value
        best_x0 = x0

# Print information about the best solution
print("Best Solution:")
print("Best Initial Point (x0):", best_x0)
print("Solution:", best_solution)
best_w = best_solution / np.sum(best_solution) if np.sum(best_solution) != 0 else best_solution
print("Best w:", best_w)
print("Expected Return:", np.dot(best_w, mean_R))
print("Risk:", np.sum(np.outer(best_w, best_w) * M))
print("Objective Function Value:", best_objective_value)

Run 1:
Initial Point (x0): [0.80745509 0.67583697 0.28866242 0.92361777]
Solution: [0.8127067  0.61757548 0.34150962 0.93743572]
w [0.29997728 0.22795261 0.12605424 0.34601587]
Expected Return: 0.48938799958165563
Risk: 0.09123555799125138
Objective Function Value: -0.35253466259477856


Run 2:
Initial Point (x0): [0.28399996 0.42850827 0.72139503 0.30156093]
Solution: [0.29236409 0.41842173 0.73217395 0.31019732]
w [0.16676434 0.23866757 0.41763168 0.17693641]
Expected Return: 0.5721898597701329
Risk: 0.12128305514824983
Objective Function Value: -0.39026527704775815


Run 3:
Initial Point (x0): [0.30818718 0.62024189 0.29363972 0.56142134]
Solution: [0.31016726 0.70862469 0.26171704 0.57096965]
w [0.16752408 0.38273447 0.14135569 0.30838576]
Expected Return: 0.4987040229880609
Risk: 0.13718140147265173
Objective Function Value: -0.29293192077908325


Run 4:
Initial Point (x0): [0.03779214 0.92335447 0.33438002 0.64657515]
Solution: [0.12197304 0.85247664 0.4073438  0.68243351]
w [0.0

In [394]:
#Μέθοδο συζυγών κλίσεων Polak-Ribiere

In [67]:
def line_search_wolfe(f, g, x, d, c1=1e-4, c2=0.9, max_iter=100):
    alpha = 0.1  # Αρχική τιμή βήματος
    phi0 = f(x)  # Τιμή της συνάρτησης στο σημείο εκκίνησης
    alpha_max = 2  # Μέγιστο επιτρεπόμενο βήμα
    phi_prime0 = np.dot(g(x), d)  # Κλίση στο σημείο εκκίνησης
    alpha_prev = 0.0
    phi_prev = phi0
        
    for i in range(max_iter):
        x_new = x + alpha * d
        phi = f(x_new)
        phi_prime = np.dot(g(x_new), d)
        
        # Έλεγχος συνθήκης Armijo (συνθήκη 1ης τάξης)
        if phi > phi0 + c1 * alpha * phi_prime0 or (i > 1 and phi >= phi_prev):
            return zoom(f, g, x, d, alpha_prev, alpha)
        
        # Έλεγχος συνθήκης Καμπυλότητας (συνθήκη 2ης τάξης)
        if abs(phi_prime) <= -c2 * phi_prime0:
            return alpha
        
        # Έλεγχος για αρνητική κλίση
        if phi_prime >= 0:
            return zoom(f, g, x, d, alpha, alpha_prev)
        
        alpha_prev = alpha
        phi_prev = phi
        alpha *= 2.0
        
        # Έλεγχος για υπέρβαση του μέγιστου επιτρεπόμενου βήματος
        if alpha > alpha_max:
            return alpha_max  # Εφαρμόζεται το `alpha_max` ως όριο
    
    return alpha

In [68]:
def zoom(f, g, x, d, alpha_lo, alpha_hi, c1=1e-4, c2=0.9, max_iter=100):
    phi0 = f(x)  # Τιμή της συνάρτησης στο σημείο εκκίνησης
    phi_prime0 = np.dot(g(x), d)  # Κλίση στο σημείο εκκίνησης
    
    for i in range(max_iter):
        alpha = 0.5 * (alpha_lo + alpha_hi)  # Υπολογισμός ενδιάμεσου βήματος
        
        x_new = x + alpha * d  # Υπολογισμός νέου σημείου
        phi = f(x_new)  # Τιμή της συνάρτησης στο νέο σημείο
        phi_prime = np.dot(g(x_new), d)  # Κλίση στο νέο σημείο
        
        # Έλεγχος συνθήκης Armijo
        if phi > phi0 + c1 * alpha * phi_prime0 or phi >= f(x + alpha_lo * d):
            alpha_hi = alpha
        else:
            # Έλεγχος συνθήκης Wolfe
            if abs(phi_prime) <= -c2 * phi_prime0:
                return alpha
            if phi_prime * (alpha_hi - alpha_lo) >= 0:
                alpha_hi = alpha_lo
            alpha_lo = alpha
        
    return alpha

In [69]:
def polak_ribiere_cg(x0, mean_R, M, lambd, max_iter=100, tol=1e-6, restart_interval=14):
    n = len(x0)
    x = x0.copy()
    g = gradient(x, mean_R, M, lambd)  # Calculate the gradient
    d = -g  # Initial search direction
    
    restart_counter = 0
    
    for _ in range(max_iter):
        if np.linalg.norm(g) < tol:
            break
        #Call line search a0=0,1 και αmax=1
        alpha = line_search_wolfe(lambda x: objective_function(x, mean_R, M, lambd),
                              lambda x: gradient(x, mean_R, M, lambd),
                              x, d)  # Initial step size
        x_new = x + alpha * d  # Update x
        #Call Bisection
        x_new = np.array([bisection(lambda x: objective_function(x, mean_R, M, lambd), 0, 1, 1e-6) if xi < 0 or xi > 1 else xi for xi in x_new])
        #Update gradient
        g_new = gradient(x_new, mean_R, M, lambd)  # Calculate the new gradient
        
        
        
        beta = max(np.dot(g_new, g_new - g) / np.dot(g, g), 0)  # Calculate the Polak-Ribiere coefficient
        
        if restart_counter >= restart_interval:
            d = -g_new  # Restart the search direction
            restart_counter = 0
        else:
            d = -g_new + beta * d  # Update the search direction
        #Update x and g
        x = x_new
        g = g_new
        restart_counter += 1
    
    return x

In [70]:
bounds = [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]
R = [R1, R2, R3, R4]
lambd = 1.5
num_initial_points = 10
results = []
used_initial_points = set()  # To keep track of used initial points

In [71]:
best_solution = None
best_objective_value = float('inf')
best_x0 = None

In [103]:
# Execute the algorithm from 10 independently and randomly selected initial points
for i in range(num_initial_points):
    while True:
        x0 = np.random.uniform(bounds[0][0], bounds[0][1], size=4)
        # Convert the array to a tuple for set comparison
        x0_tuple = tuple(x0)
        if x0_tuple not in used_initial_points:
            used_initial_points.add(x0_tuple)
            break
    
    result = polak_ribiere_cg(x0, mean_R, M, lambd)
    results.append(result)
    # Calculate the total sum and weight vector
    total_sum = np.sum(result)
    w = result / total_sum if total_sum != 0 else x

    # Print information for each run
    print(f"Run {i + 1}:")
    print("Initial Point (x0):", x0)
    print("Solution:", result)
    print ("w", w)
    print("Expected Return:", np.dot(w, mean_R))
    print("Risk:", np.sum(np.outer(w, w) * M))
    objective_value = -np.dot(w, mean_R) + lambd * np.sum(np.outer(w, w) * M)
    print("Objective Function Value:", objective_value)
    
    print("\n")

    # Check if the current solution is better than the previous best
    if objective_value < best_objective_value:
        best_solution = result
        best_objective_value = objective_value
        best_x0 = x0

# Print information about the best solution
print("Best Solution:")
print("Best Initial Point (x0):", best_x0)
print("Solution:", best_solution)
best_w = best_solution / np.sum(best_solution) 
print("Best w:", best_w)
print("Expected Return:", np.dot(best_w, mean_R))
print("Risk:", np.sum(np.outer(best_w, best_w) * M))
print("Objective Function Value:", best_objective_value)

Run 1:
Initial Point (x0): [0.21607839 0.66335072 0.93836558 0.01463772]
Solution: [0.34710039 0.15248836 0.99999952 0.31120721]
w [0.19168393 0.0842107  0.55224322 0.17186215]
Expected Return: 0.6196965530506415
Risk: 0.11503535290567588
Objective Function Value: -0.44714352369212773


Run 2:
Initial Point (x0): [0.12741271 0.61849273 0.4399686  0.01174333]
Solution: [0.34829366 0.02540878 0.99999952 0.32540787]
w [0.20498596 0.01495417 0.58854319 0.19151668]
Expected Return: 0.6369085938534946
Risk: 0.1153315018255665
Objective Function Value: -0.46391134111514487


Run 3:
Initial Point (x0): [0.1680117  0.11019757 0.94655449 0.881484  ]
Solution: [0.43541432 0.3311419  0.99999952 0.45762807]
w [0.19576364 0.14888244 0.44960291 0.20575101]
Expected Return: 0.5884664783596918
Risk: 0.11027887974577291
Objective Function Value: -0.4230481587410324


Run 4:
Initial Point (x0): [0.67314527 0.63866271 0.66988317 0.02068424]
Solution: [0.35714098 0.01379778 0.99999952 0.31823352]
w [0.2114